# Installations and Import statements

In [ ]:
# !pip3 install -q transformers sagemaker-inference
!pip install boto3 sagemaker -U -q
# !pip install torch==2.0.1 torchvision==0.15.2 transformers==4.31.0 -q

In [ ]:
# !tar -czvf model.tar.gz trained_model/v9-small/* 

In [ ]:
!rm  sources.tar.gz 
!tar -czvf sources.tar.gz ./inference.py

In [2]:
import boto3
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker
import json

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


# Upload Artifacts

In [ ]:
s3 = boto3.client(
    's3'
)

In [ ]:
bucket_name = 'cc-ai-bucket'
model_file = 'model.tar.gz'  
script_file = 'inference.py'
source_file = 'sources.tar.gz'

# Upload model and script
s3.upload_file(model_file, bucket_name, 'models/tag-classification/models/model.tar.gz')
s3.upload_file(script_file, bucket_name, 'models/tag-classification/scripts/inference.py')
s3.upload_file(source_file, bucket_name, 'models/tag-classification/scripts/sources.tar.gz')

# Sagemaker Role 
Needs SagemakerFullAccess policy to deploy get access to EC2 instances and deploy models

In [5]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

print(f"sagemaker role arn: {role}")

sagemaker role arn: arn:aws:iam::767397973834:role/service-role/AmazonSageMaker-ExecutionRole-20240809T105831


In [6]:
sagemaker_session = sagemaker.Session(
)

# Specify Artifact Location
Location where model and source artifacts are uploaded 

In [7]:
bucket_name = "cc-ai-bucket/models/tag-classification"
model_artifact = f's3://{bucket_name}/models/model.tar.gz'
script_location = f's3://{bucket_name}/scripts/'
print(model_artifact)

s3://cc-ai-bucket/models/tag-classification/models/model.tar.gz


# Deploy Hugging Face Model

In [8]:
# Initialize the Hugging Face model
huggingface_model = HuggingFaceModel(
    model_data=model_artifact,
    role=role,
    source_dir="s3://cc-ai-bucket/models/tag-classification/scripts/sources.tar.gz",
    entry_point="inference.py",  # Path to your custom inference script
    transformers_version="4.37.0",
    pytorch_version="2.1.0",
    py_version="py310",
    sagemaker_session=sagemaker_session
)

In [9]:


# Deploy the model
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.xlarge" # Can be changed g4 or g5 gpus 
)

-------!

In [10]:
print(predictor.endpoint_name)

huggingface-pytorch-inference-2024-09-06-06-14-50-836


# Test Inference

In [11]:
import pandas as pd

In [12]:
df = pd.read_json("./data/cleaned_tags_data_na_tag_lt3.json", orient='records')

In [13]:
sample_df = df[df.tags != "NA"].sort_values(by="word_count")
sample_df.head()

,title,category,tags,checklists,section_content,filename,word_count,tag_title
368,Mandatory Redemption,FYI,Mandatory Prepayments / Redemption,[],Title is Mandatory Redemption\nCategory is FYI...,Paramount Global - INDENTURE 7.250- SENIOR NOT...,24,Mandatory Prepayments / RedemptionMandatory Re...
177,Maturity,FYI,Loan Repayment,[],Title is Maturity\nCategory is FYI\nSection 8....,"MGEE - NPA Dated July 25, 2017.section.json",25,Loan RepaymentMaturity
941,Maturity,FYI,Loan Repayment,[],Title is Maturity\nCategory is FYI\nSection 8....,"MGEE - Note Purchase Agreement Aug 31, 2023.se...",25,Loan RepaymentMaturity
192,Financial Condition Covenant,Negative Covenants,Financial Covenant,[],Title is Financial Condition Covenant\nCategor...,TR_V3_Sustainability_AVANGRID.section.json,25,Financial CovenantFinancial Condition Covenant
419,Consolidated Capitalization Ratio,Affirmative Covenants,Financial Covenant,[],Title is Consolidated Capitalization Ratio\nCa...,Edison International - TERM LOAN CREDIT AGREEM...,26,Financial CovenantConsolidated Capitalization ...


In [ ]:
test_data = {
    "inputs": sample_df.sort_values(by="word_count", ascending=False).iloc[0].section_content
}
print(test_data)

In [15]:
import time 

In [16]:
start_time = time.time()
for t in sample_df.sample(20).itertuples(): 
    test_data = {
        "inputs": t.section_content
    }
    result = predictor.predict(json.dumps(test_data))
    result = json.loads(result)
    print(result[0], t.tags, result[0] == t.tags)
print("Time taken ", time.time() - start_time)

Event of Default Event of Default True
Financial Covenant Financial Covenant True
Reporting Requirements Financial Statements False
Waterfall of Proceeds Waterfall of Proceeds True
Amortization Schedule / Loan Repayment Loan Repayment False
Reporting Requirements Reporting Requirements True
Amortization Schedule / Loan Repayment Loan Repayment False
Permitted Indebtedness Permitted Indebtedness True
Premium and Fees Premium and Fees True
Financial Covenant Financial Covenant True
Reporting Requirements Reporting Requirements True
Financial Covenant Financial Covenant True
Consequences of Default Consequences of Default True
Amortization Schedule / Loan Repayment Loan Repayment False
Amortization Schedule / Loan Repayment Loan Repayment False
Compliance Certificate Reporting Requirements False
NA Facilities / Instrument False
Restricted Payments Restricted Payments True
Facilities / Instrument Facilities / Instrument True
Incremental Facilities Incremental Facilities True
Time taken  9.

In [17]:
7.87/20

0.3935

In [22]:
result = predictor.predict(json.dumps(test_data))
print(f"Prediction: {result}")

Prediction: ["Facilities / Instrument"]


# Test Inference from Client 

In [40]:

# Initialize the SageMaker runtime client
runtime_client = boto3.client('sagemaker-runtime')

# Define the endpoint name
endpoint_name = "huggingface-pytorch-inference-2024-09-06-06-14-50-836"

# Convert input data to JSON string
payload = json.dumps(test_data)
# Invoke the endpoint
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=payload
)
# Parse the response
result = json.loads(response['Body'].read().decode())
print(f"Prediction: {result}")

Prediction: ["Mandatory Prepayments / Redemption"]


In [40]:
print("Deleting Model")
predictor.delete_model()
print("Deleting Endpoint")
predictor.delete_endpoint()


Deleting Model
Deleting Endpoint
